# Fine-Tuning BERT for Named Entity Recognition

In [1]:
# import relevant packages.
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

### Check availability of GPU

In [2]:
print(torch.cuda.is_available())

True


### Get training and test dataset

In [ ]:
label_list = ['O','B-DOMMER','I-DOMMER','B-ADVOKAT','I-ADVOKAT','B-LOV','I-LOV','B-RET','I-RET','B-PERSON','I-PERSON','B-ORGANISATION','I-ORGANISATION','B-LOKATION','I-LOKATION','B-DATO','I-DATO','B-SAGSEMNE','I-SAGSEMNE']
label_encoding_dict = {'O': 0, 'B-DOMMER': 1, 'I-DOMMER': 2, 'B-ADVOKAT': 3, 'I-ADVOKAT': 4, 'B-LOV': 5, 'I-LOV': 6, 'B-RET': 7, 'I-RET': 8, 'B-PERSON': 9, 'I-PERSON': 10, 'B-ORGANISATION': 11,'I-ORGANISATION': 12, 'B-LOKATION': 13, 'I-LOKATION': 14, 'B-DATO': 15, 'I-DATO': 16, 'B-SAGSEMNE': 17, 'I-SAGSEMNE': 18}

task = "ner" 
model_checkpoint = "distilbert-base-multilingual-cased"
batch_size = 16
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    
def get_tokens_and_ner_tags(filename):
    with open(filename, 'r', encoding="utf-8") as f:
        lines = f.readlines()
        split_list = [list(y) for x, y in itertools.groupby(lines, lambda z: z == '\n') if not x]
        tokens = [[x.split(' ')[0] for x in y] for y in split_list]
        entities = []
        for y in split_list:
            doc = []
            for x in y:
                try:
                    doc.append(x.split(' ')[3][:-1])
                except IndexError:
                    doc.append(x.split(' ')[2][:-1])
            entities.append(doc)

    return pd.DataFrame({'tokens': tokens, 'ner_tags': entities})
  
def get_token_dataset(filename):
    df = get_tokens_and_ner_tags(filename)
    dataset = Dataset.from_pandas(df)
    dataset = dataset.train_test_split(test_size=0.2)
    train_dataset = dataset['train']
    test_dataset = dataset['test']

    return (train_dataset, test_dataset)

train_dataset, test_dataset = get_token_dataset('../data/danish_legal_ner_dataset.conll')

### Tokenize datasets.

In [ ]:
def tokenize_and_align_labels(examples):
    label_all_tokens = False
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

### Fine-Tune BERT model for Named Entity Recognition.

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=test_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model('danish-ner-model')

### Predict entities

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('danish-ner-model')

paragraph = '''Den 11. april 2014 holdt Østre Landsret møde i retsbygningen, Bredgade 59, København. Som dommere fungerede landsdommerne Mogens Kroman, Betina Juul Heldmann (kst.) og Andreas Bøgsted-Møller (kst.), førstnævnte som rettens formand. Som protokolfører fungerede Medarbejder ved retten. Der foretoges 5. afd. kære nr. B-975-14: Kærende, tidligere Far mod Indkærede, tidligere Mor Ingen var mødt eller indkaldt. Der fremlagdes kæreskrift af 29. marts 2014, hvorved Kærende, tidligere Far har påkæret Odense Rets kendelse af 24. marts 2014 (BS 3-390/2014), hvor det tiltrædes, at statsfor-valtningen har afvist at behandle Kærendes, tidligere Far anmodning om overførelse af for-ældremyndigheden over Barn 1 til sig, dommerens fremsendelsesbrev af 1. april 2014 og udskrift af retsbogen med den påkærede afgørelse. Indkærede har ikke udtalt sig vedrørende kæren. Byretten har ved sagens fremsendelse henholdt sig til den trufne afgørelse. De modtagne bilag var til stede. Efter votering afsagdes k e n d e l s e : Landsretten er enig i byrettens resultat og begrundelsen herfor. Landsretten stadfæster derfor byrettens afgørelse. T h i b e s t e m m e s : Byrettens kendelse stadfæstes. Ingen af parterne skal betale kæremålsomkostninger til den anden part. Retten hævet. '''
tokens = tokenizer(paragraph)
torch.tensor(tokens['input_ids']).unsqueeze(0).size()

model = AutoModelForTokenClassification.from_pretrained('danish-ner-model', num_labels=len(label_list))
predictions = model.forward(input_ids=torch.tensor(tokens['input_ids']).unsqueeze(0), attention_mask=torch.tensor(tokens['attention_mask']).unsqueeze(0))
predictions = torch.argmax(predictions.logits.squeeze(), axis=1)
predictions = [label_list[i] for i in predictions]

words = tokenizer.batch_decode(tokens['input_ids'])
pd.DataFrame({'ner': predictions, 'words': words})
pd.DataFrame({'ner': predictions, 'words': words}).to_csv('danish_legal_ner.csv')